# IMDB Datasets

**Purpose:**

Working with "big" data in PySpark

**Data Source:**
    
https://datasets.imdbws.com

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; font-size:90%;}</style>"))

In [ ]:
import sys

print('Python info', sys.version)

In [ ]:
import os

print('This is the curent directory', os.getcwd())

In [ ]:
import datetime

current_date = datetime.date.today()
current_time = datetime.datetime.now()

print("System date/time", current_time)
print('Current date', datetime.datetime.strftime(current_date, '%A %m/%d/%Y'))
print('Current time', datetime.datetime.strftime(current_time, '%I:%M:%S %p'))

In [ ]:
import requests
from bs4 import BeautifulSoup

from pprint import pprint
import re

In [ ]:
!pip install pyspark --upgrade

In [ ]:
!pip install pyarrow --upgrade

In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("Spark SQL Query Dataframes").getOrCreate()
spark

In [ ]:
#spark = SparkSession.builder.appName("IMDB_Movies")\
#.config("spark.sql.shuffle.partitions", "50")\
#.config("spark.driver.maxResultSize", "5g")\
#.config("Spark.sql.execution.arrow.enabled", "true")\
#.getOrCreate()

#spark

In [ ]:
sc = spark.sparkContext

In [ ]:
!pip install wget

In [ ]:
import wget

import gzip
import shutil
import csv

In [ ]:
URL = "https://datasets.imdbws.com/title.basics.tsv.gz"

In [ ]:
print(URL)

In [ ]:
filename = wget.download(URL)

In [ ]:
print(filename)

In [ ]:
with gzip.open(filename, mode='r') as f_in, open(filename.replace('.gz',''), mode='wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
    
uncompressed_file = filename.replace('.gz','')

print(uncompressed_file)

In [ ]:
# JSON
#df = sc.read.json('fp.json')

# TXT
#df = sc.read.text('fp.txt')

# CSV
#df = sc.read_csv('fp.csv')

In [ ]:
spark_df0 = spark.read.format("csv")\
    .option("header","true")\
    .option("sep","\t")\
    .option("encoding","utf-8")\
    .option("nullValues", r"\N")\
    .option("quote","")\
    .option("inferSchema","true")\
    .load(uncompressed_file)

spark_df0.show(truncate=False)

In [ ]:
spark_df0.printSchema()

In [ ]:
spark_df0.dtypes

In [ ]:
spark_df0.columns

In [ ]:
spark_df0.head()

In [ ]:
spark_df0.take(15)

In [ ]:
spark_df0.groupBy(['isAdult','titleType']).count().sort(['isAdult','titleType']).show()

In [ ]:
spark_df0 = spark_df0.filter((spark_df0["isAdult"]!=1) & (spark_df0["titleType"] == 'movie'))

In [ ]:
spark_df0.groupBy(['isAdult','titleType']).count().sort(['isAdult','titleType']).show()

In [ ]:
URL = "https://datasets.imdbws.com/title.ratings.tsv.gz"
filename = wget.download(URL)

In [ ]:
with gzip.open(filename, mode='r') as f_in, open(filename.replace('.gz',''), mode='wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
    
uncompressed_file = filename.replace('.gz','')

print(uncompressed_file)

In [ ]:
spark_df1 = spark.read.format("csv")\
    .option("header","true")\
    .option("sep","\t")\
    .option("encoding","utf-8")\
    .option("nullValues", r"\N")\
    .option("quote","")\
    .option("inferSchema","true")\
    .load(uncompressed_file)

spark_df1.show()

In [ ]:
spark_df1.printSchema()

In [ ]:
spark_df = spark_df0.join(spark_df1, on='tconst', how='inner')

In [ ]:
spark_df.show(truncate=False)

In [ ]:
spark_df.createOrReplaceTempView('imdb_data')

In [ ]:
spark.sql("SELECT * FROM imdb_data LIMIT 10").show(truncate=False)

In [ ]:
spark.sql("SELECT titleType, startYear, count(*) totals FROM imdb_data GROUP BY titleType, startYear ORDER BY 1, 2").show(truncate=False)

In [ ]:
stmt = \
"""
SELECT tconst, titleType, primaryTitle, startYear, runtimeMinutes, averageRating, numVotes
FROM imdb_data
WHERE primaryTitle LIKE '%Star Wars%'
"""

spark.sql(stmt).show()

In [ ]:
stmt = """
SELECT *
FROM imdb_data
WHERE
    titleType = 'movie' AND
    averageRating > 9 AND
    genres != '\\N' AND
    numVotes > 1000
ORDER BY
    averageRating DESC
"""

spark.sql(stmt).show(25)

In [ ]:
stmt = """
SELECT *
FROM imdb_data
WHERE
    titleType = 'movie' AND
    averageRating >= 7 AND
    genres LIKE '%Action%' AND
    CAST(runtimeMinutes AS INT) > 30 AND
    CAST(startYear AS INT) BETWEEN 2015 and 2019   
ORDER BY
    startYear DESC, averageRating DESC
"""

spark.sql(stmt).show(n=50, truncate=False)

In [ ]:
URL = 'https://m.imdb.com/chart/top'

response = requests.get(URL)
soup = BeautifulSoup(response.text, 'lxml')

soup.title

In [ ]:
soup.h1

In [ ]:
soup.h2

In [ ]:
soup.h3

In [ ]:
soup.h4

In [ ]:
soup.find_all('a')

In [ ]:
data = []

for link in soup.find_all("a"):
    data.append(link.get("href", 'Empty'))
    
text_data = " ".join(data)

In [ ]:
movie_titles = re.findall(pattern="(tt\d+)", string=text_data)

pprint(set(movie_titles), compact=True, width=132)
print()
print(len(set(movie_titles)))

In [ ]:
spark_top250 = sc.parallelize([Row(tconst=t) for t in set(movie_titles)]).toDF()

In [ ]:
spark_df = spark_top250.join(spark_df0, on='tconst', how='inner')
spark_df = spark_df.join(spark_df1, on='tconst', how='inner')
spark_df.show()

In [ ]:
spark_df.drop_duplicates().show(n=250, truncate=False)

In [ ]:
import pandas as pd

In [ ]:
pandas_top250 = spark_df.toPandas()

In [ ]:
pandas_top250.info()

In [ ]:
pandas_top250.head()

In [ ]:
pandas_top250['runtimeMinutes'] = pd.to_numeric(pandas_top250['runtimeMinutes'])

In [ ]:
pandas_top250.runtimeMinutes.plot.hist(alpha=.4);

In [ ]:
pandas_top250.query("runtimeMinutes > 200")

In [ ]:
pandas_top250.startYear.value_counts().sort_index().plot.bar(figsize=(14,4), alpha=.4);

In [ ]:
pandas_top250['startYear'] = pandas_top250['startYear'].astype(int)

In [ ]:
pandas_top250.query("startYear == 1995")

In [ ]:
pandas_top250.plot.scatter(x='averageRating', y='numVotes', alpha=.5);

In [ ]:
pandas_top250.query("averageRating > 9 & numVotes > 1.5e6")